In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

close = pd.read_parquet("Adj_close.parquet")
df = close.loc['2020-01-01':]

In [9]:
ma20 = df.rolling(window=int(2)).mean()
diff_ma20 = df/ma20 - 1
diff_ma20.dropna(how='all', axis=0, inplace=True)
# def get_demean_weighting(self, data):
df1 = diff_ma20.dropna(axis='columns', how='all').copy()
demean = df1.sub(df1.mean(axis=1), axis=0)
weighting = demean.div(demean.abs().sum(axis=1), axis=0)
# 計算出來的因子（diff_ma20）是當天計算出來的，所以要再下一個交易日Open/Current Close買進
